# MDN and Random Forest Comparison

## Prepare Data

In [2]:
import tensorflow as tf
import numpy as np
import mdn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import pandas as pd
from timeit import default_timer as timer
import xgboost as xgb

bin_count = 171

Split our dataset into a train and test set

In [3]:
def find_first_zero(row):
    vals = list(row) 
  
    last_zero_index = -1
    for idx, val in enumerate(reversed(vals)): 
        if val > 0:
            last_zero_index = bin_count - idx
            break
    return last_zero_index

def create_test_train(data_set_path, test_size=0.10):
    """ Splits a given csv file into testing and training. Target column is all the bins. Add a c """
    # Make sure the columns are set
    data_set = pd.read_csv(data_set_path)

    # Add column for classifying whether the output has most of the data in the last 10 bins.
    end_average = data_set[[f'Output_Bin_{i}' for i in range(bin_count-10, bin_count)]].sum(axis=1) > 0.9

    data_set['Output_Is_End'] = end_average
    data_set['Output_Is_End'] = data_set['Output_Is_End'].astype(int)
    #data_set['Output_Is_Not_End'] = ~data_set['Output_Is_End']
    
    

    data_set['Output_First_Zero'] = data_set[[f'Output_Bin_{i}' for i in range(bin_count)]].apply(find_first_zero, axis=1)

    # Shuffle the data
    data_set = data_set.sample(frac=1, random_state=0)
 
    # Select all except output bins
    data_set_X = data_set.drop([f'Output_Bin_{i}' for i in range(bin_count)] + ['Output_Is_End', 'Output_First_Zero'], axis=1)
    # Select only the output bins
    data_set_Y = data_set[[f'Output_Bin_{i}' for i in range(bin_count)]+ ['Output_Is_End', 'Output_First_Zero']]

    #Split into training and test data
    return train_test_split(data_set_X,
                            data_set_Y,
                            test_size=test_size, 
                            random_state=300)

#filename = "/scratch/keh4nb/dust_training_data_all_bins_large.csv"
filename= "/project/SDS-capstones-kropko21/uva-astronomy/dust_training_data_all_bins_v2.csv"
X_train, X_test, y_train, y_test = create_test_train(filename, test_size=0.10)
display(y_test.describe())

,Output_Bin_0,Output_Bin_1,Output_Bin_2,Output_Bin_3,Output_Bin_4,Output_Bin_5,Output_Bin_6,Output_Bin_7,Output_Bin_8,Output_Bin_9,...,Output_Bin_163,Output_Bin_164,Output_Bin_165,Output_Bin_166,Output_Bin_167,Output_Bin_168,Output_Bin_169,Output_Bin_170,Output_Is_End,Output_First_Zero
count,1.423300e+05,1.423300e+05,1.423300e+05,1.423300e+05,1.423300e+05,1.423300e+05,1.423300e+05,1.423300e+05,1.423300e+05,1.423300e+05,...,142330.000000,142330.000000,142330.000000,142330.000000,142330.000000,142330.000000,142330.000000,142330.000000,142330.000000,142330.000000
mean,4.788404e-03,3.511234e-03,3.102781e-03,3.476701e-03,3.457728e-03,3.501525e-03,3.642096e-03,3.778553e-03,3.935451e-03,4.021584e-03,...,0.001058,0.000883,0.000728,0.000676,0.001939,0.005278,0.016270,0.151275,0.167596,114.466395
std,1.166852e-02,7.803030e-03,6.573299e-03,7.853398e-03,6.781768e-03,6.542634e-03,6.904298e-03,6.660142e-03,7.485042e-03,6.836560e-03,...,0.006218,0.005424,0.004790,0.004623,0.006232,0.013469,0.040799,0.326542,0.373509,41.073065
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,26.000000
25%,1.413419e-11,5.957408e-11,2.004835e-10,7.735218e-10,2.203720e-09,4.972974e-09,1.233129e-08,2.641020e-08,5.567167e-08,1.080826e-07,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,81.000000
50%,1.091139e-04,1.224256e-04,1.354748e-04,1.791490e-04,2.160863e-04,2.491617e-04,2.911052e-04,3.537974e-04,4.251557e-04,5.010170e-04,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,110.000000
75%,2.666961e-03,2.534129e-03,2.509944e-03,2.984808e-03,3.315966e-03,3.667415e-03,4.060019e-03,4.502340e-03,4.900773e-03,5.306905e-03,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,157.000000
max,9.354140e-02,5.693502e-02,4.501455e-02,3.978454e-01,1.493864e-01,3.745879e-02,2.716096e-01,3.748155e-02,3.097229e-01,1.159227e-01,...,0.087176,0.087361,0.087655,0.091350,0.105380,0.133252,0.245536,0.999991,1.000000,171.000000


## MDN Training

Train 2 MDN models, one on where the most of the dust is at the end and the other where its not

Helper function for creating a MDN model

In [13]:
def build_mdn_model(activation='sigmoid', k=16):
    """ k is the number of mixture models"""
    # Number of columns (bin count plus 8 inputs)
    l = bin_count + 8

    # Network
    input = tf.keras.Input(shape=(l,))

    layer = tf.keras.layers.Dense(512, activation=activation, name='baselayer')(input)
    layer_2 = tf.keras.layers.Dense(128, activation=activation, name='layer_2')(layer)
    layer_3 = tf.keras.layers.Dense(64, activation=activation, name='layer_3')(layer_2)
    #layer_4 = tf.keras.layers.Dense(32, activation=activation, name='layer_4')(layer_3)

    # Connect the mdn layer to the output of our neural network
    mdn_layer = mdn.MDN(bin_count,k, name='mdn')(layer_3)
    model = tf.keras.models.Model(input, [mdn_layer])


    opt = tf.keras.optimizers.Adam()
    model.compile(loss=mdn.get_mixture_loss_func(bin_count,k), optimizer=opt)
    model.summary()
    return model

Create a MDN model on the data where most of the dust is at the end of the distribution

In [5]:

#model_spike = build_mdn_model(k=64)
model_spike = build_mdn_model(k=32)
output_is_end_idx = y_train.Output_Is_End.index[y_train['Output_Is_End'] == 1]

X = X_train.loc[output_is_end_idx]
y = y_train.loc[output_is_end_idx].drop(['Output_Is_End', 'Output_First_Zero'], axis=1)

spike_fit = model_spike.fit(x=X, y=y, batch_size=512, epochs=50, validation_split=0.1, callbacks=[tf.keras.callbacks.TerminateOnNaN()])
model_spike.save("spiked_mdn_model/model")

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 179)]             0         
_________________________________________________________________
baselayer (Dense)            (None, 512)               92160     
_________________________________________________________________
layer_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
layer_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
mdn (MDN)                    (None, 10976)             713440    
Total params: 879,520
Trainable params: 879,520
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
377/377 [=======

Define a function to make predections from the MDN output

In [14]:
from scipy.stats import entropy
def predict_for_test(model, X_test, y_test, k):

    start = timer()

    # Returns the predictions of the parameters of the distributions and weights
    preds = model.predict(X_test)
    samples_list = []
    # Obtain 5 samples per prediction
    for i in range(5):
        samples_list.append(np.apply_along_axis(mdn.sample_from_output, 1, preds, bin_count, k, temp=1.0, sigma_temp=1.0))

    # Average the samples for our predicitons
    y_samples = np.mean(np.array(samples_list),axis=0)
    y_samples = y_samples.reshape(len(X_test),bin_count)
    # Convert negatives to 0
    y_samples = np.clip(y_samples,0, a_max=None)
    end = timer()
    
    print(f"Total time taken: {end-start}. Seconds per prediction: {(end-start)/len(preds)}")
    
    return y_samples

Load the model and make predictions

In [16]:
output_is_end_idx = y_test.Output_Is_End.index[y_test['Output_Is_End'] == 1]

X_test_spike = X_test.loc[output_is_end_idx]
y_test_spike = y_test.loc[output_is_end_idx].drop(['Output_Is_End','Output_First_Zero'], axis=1)

model_spike = tf.keras.models.load_model('spiked_mdn_model/model', custom_objects={'MDN': mdn.MDN, 'mdn_loss_func': mdn.get_mixture_loss_func(1, 32)})


spike_preds = predict_for_test(model_spike, X_test_spike, y_test_spike, 32)
#spike_preds = model_spike.predict(X_test)

Total time taken: 933.9000097643584. Seconds per prediction: 0.039150666964213904


Define a function that evaluates the fit using the entorpy and Jenson-Shannon distance

In [17]:
from scipy.stats import entropy
from scipy.spatial.distance import jensenshannon
# Renormalize samples
spike_preds_normalized = []
for s in spike_preds:
    spike_preds_normalized.append(np.divide(s,np.sum(s)))

def evaluate_fit(y_samples, y_test):
    ent = []
    js_list = []

    # Turn all negative preds to 0
    y_samples = np.clip(y_samples,0, a_max=None)

    #y_test_obs = [obs_from_bins(s.array) for idx, s in y_test.iterrows()]
    #y_samples_obs = [obs_from_bins(s) for s in y_samples]
    
    y_samples_obs = y_samples
    y_test_obs = y_test
    
    # Small constant to prevent inf for 0s
    c = 1e-100
    
    y_test_obs += c
    y_samples_obs += c

    #print(y_test_obs)
    for i in range(len(y_test_obs)):
        e = entropy(y_test_obs.iloc[i], y_samples_obs[i])
        js = jensenshannon(y_test_obs.iloc[i], y_samples_obs[i])
        ent.append(e if e != np.inf else 1000)
        js_list.append(js)
    display(pd.DataFrame(ent).describe())
    display(pd.DataFrame(js_list).describe())
    return ent, js_list

In [18]:
mdn_spike_ent, mdn_spike_js = evaluate_fit(spike_preds_normalized, y_test_spike)

,0
count,23854.000000
mean,0.603968
std,1.875910
min,0.047819
25%,0.073872
50%,0.100202
75%,0.233044
max,33.370230


,0
count,23854.000000
mean,0.199211
std,0.066228
min,0.127983
25%,0.153333
50%,0.166709
75%,0.249018
max,0.636987


Create a second MDN model for the data not at the end

In [19]:
model_non_spike = build_mdn_model()

output_is_not_end_idx = y_train.Output_Is_End.index[y_train['Output_Is_End'] == 0]

X = X_train.loc[output_is_not_end_idx]
y = y_train.loc[output_is_not_end_idx].drop(['Output_Is_End', 'Output_First_Zero'], axis=1)
non_spike_fit = model_non_spike.fit(x=X, y=y, batch_size=1024, epochs=30, validation_split=0.1, callbacks=[tf.keras.callbacks.TerminateOnNaN()])

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 179)]             0         
_________________________________________________________________
baselayer (Dense)            (None, 512)               92160     
_________________________________________________________________
layer_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
layer_3 (Dense)              (None, 64)                8256      
_________________________________________________________________
mdn (MDN)                    (None, 5488)              356720    
Total params: 522,800
Trainable params: 522,800
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
Instructions for updating:
Do not pass `graph_parents`.  They will  no longer be used.
938/938 [=======

In [15]:
model_non_spike.save("non_spiked_mdn_model/model")

INFO:tensorflow:Assets written to: non_spiked_mdn_model/model/assets


In [20]:
#model_non_spike = tf.keras.models.load_model('non_spiked_mdn_model/model', custom_objects={'MDN': mdn.MDN, 'mdn_loss_func': mdn.get_mixture_loss_func(1, 32)})

output_is_not_end_idx = y_test.Output_Is_End.index[y_test['Output_Is_End'] == 0]

X_test_non_spike = X_test.loc[output_is_not_end_idx]
y_test_non_spike = y_test.loc[output_is_not_end_idx].drop(['Output_Is_End','Output_First_Zero'], axis=1)

non_spike_preds = predict_for_test(model_non_spike, X_test_non_spike, y_test_non_spike, 16)

Total time taken: 3608.140143165365. Seconds per prediction: 0.0304546080485952


In [23]:
# Renormalize samples
non_spike_preds_normalized = []
for s in non_spike_preds:
    non_spike_preds_normalized.append(np.divide(s,np.sum(s)))
mdn_non_spike_ent, mdn_non_spike_js = evaluate_fit(non_spike_preds_normalized, y_test_non_spike)

,0
count,118476.000000
mean,19.905145
std,26.719506
min,0.193024
25%,3.041318
50%,9.700957
75%,24.364192
max,211.718343


,0
count,118476.000000
mean,0.537990
std,0.095873
min,0.180979
25%,0.471375
50%,0.533779
75%,0.602889
max,0.827811


## Random Forest Training

Fit a random forest model on the same spiked and non spiked data

In [ ]:
from sklearn.ensemble import RandomForestRegressor
output_is_end_train_idx = y_train.Output_Is_End.index[y_train['Output_Is_End'] == 1]


X_end = X_train.loc[output_is_end_train_idx]
y_end = y_train.loc[output_is_end_train_idx].drop(['Output_Is_End', 'Output_First_Zero'], axis=1)

# Fit and build a random forest regression model on the data with the spike at the end
rf_spike = RandomForestRegressor(max_depth=12, n_estimators=500, n_jobs=8, verbose=2, random_state=0)
rf_spike.fit(X_end, y_end)

#from joblib import dump, load
#dump(rf_spike, 'rf-spike-model-comparison.joblib') 

In [26]:
#rf_spike = load('rf-spike-model-comparison.joblib')
rf_spike_preds = rf_spike.predict(X_test_spike)


[Parallel(n_jobs=8)]: Done  25 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 146 tasks      | elapsed:    1.2s
[Parallel(n_jobs=8)]: Done 349 tasks      | elapsed:    2.7s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    3.9s finished


In [36]:
rf_spike_preds_normalized = []
for s in rf_spike_preds:
    s = s[:-1]
    rf_spike_preds_normalized.append(np.divide(s,np.sum(s)))
rf_spike_ent, rf_spike_js = evaluate_fit(rf_spike_preds_normalized, y_test_spike)

,0
count,2.385400e+04
mean,1.473415e-03
std,9.761221e-03
min,5.969657e-08
25%,6.902824e-05
50%,1.484319e-04
75%,5.408570e-04
max,4.993669e-01


,0
count,23854.000000
mean,0.011923
std,0.014940
min,0.000122
25%,0.004669
50%,0.006827
75%,0.012765
max,0.253287


Fit a second random forest model on the non spiked data

In [ ]:
from joblib import dump, load

output_is_not_end_train_idx = y_train.Output_Is_End.index[y_train['Output_Is_End'] == 0]

X_not_end = X_train.loc[output_is_not_end_train_idx]
y_not_end = y_train.loc[output_is_not_end_train_idx].drop(['Output_Is_End'], axis=1)

from sklearn.ensemble import RandomForestRegressor

#rf_non_spike = RandomForestRegressor(max_depth=20, n_estimators=1000, n_jobs=8, verbose=2, random_state=0)
#rf_non_spike.fit(X_not_end, y_not_end)
rf_non_spike = load('/project/SDS-capstones-kropko21/uva-astronomy/rf-model-depth-20-trees-1000-min_samps-32.joblib')
rf_non_spike_preds = rf_non_spike.predict(X_test_non)


In [8]:
rf_non_spike_preds = rf_non_spike.predict(X_test_non_spike)

In [12]:
rf_non_spike_preds_normalized = []
for s in rf_non_spike_preds:
    rf_non_spike_preds_normalized.append(np.divide(s,np.sum(s)))
rf_non_spike_ent, rf_non_spike_js = evaluate_fit(rf_non_spike_preds_normalized, y_test_non_spike)

,0
count,1.184760e+05
mean,3.594791e-02
std,1.322091e-01
min,5.396603e-13
25%,1.940351e-05
50%,4.534408e-04
75%,9.187106e-03
max,5.635514e+00


,0
count,1.184760e+05
mean,4.774171e-02
std,8.726648e-02
min,3.673692e-07
25%,2.241770e-03
50%,1.132620e-02
75%,5.058478e-02
max,7.867705e-01


# Comparison

Compare the entropy and JS result of the Mixture Density Network and Random Forest Regression on the data where there is a spike at the end.

In [43]:
df_spike_results = pd.DataFrame({
    "MDN Spike Entropy":mdn_spike_ent, 
    "RF Spike Entropy":rf_spike_ent, 
    "MDN Spike JS": mdn_spike_js,
    "RF SPike JS": rf_spike_js
})
display(df_spike_results.describe().apply(lambda s: s.apply('{0:.4f}'.format)))

,MDN Spike Entropy,RF Spike Entropy,MDN Spike JS,RF SPike JS
count,23854.0000,23854.0000,23854.0000,23854.0000
mean,0.6040,0.0015,0.1992,0.0119
std,1.8759,0.0098,0.0662,0.0149
min,0.0478,0.0000,0.1280,0.0001
25%,0.0739,0.0001,0.1533,0.0047
50%,0.1002,0.0001,0.1667,0.0068
75%,0.2330,0.0005,0.2490,0.0128
max,33.3702,0.4994,0.6370,0.2533


Compare the entropy and JS result of the Mixture Density Network and Random Forest Regression on the data where there is not a spike at the end.

In [44]:
df_non_spike_results = pd.DataFrame({
    "MDN No Spike Entropy":mdn_non_spike_ent,
    "RF No Spike Entropy":rf_non_spike_ent, 
    "MDN No Spike JS": mdn_non_spike_js,
    "RF No Spike JS": rf_non_spike_js
})
display(df_non_spike_results.describe().apply(lambda s: s.apply('{0:.4f}'.format)))

,MDN No Spike Entropy,RF No Spike Entropy,MDN No Spike JS,RF No Spike JS
count,118476.0000,118476.0000,118476.0000,118476.0000
mean,19.9051,0.0359,0.5380,0.0477
std,26.7195,0.1322,0.0959,0.0873
min,0.1930,0.0000,0.1810,0.0000
25%,3.0413,0.0000,0.4714,0.0022
50%,9.7010,0.0005,0.5338,0.0113
75%,24.3642,0.0092,0.6029,0.0506
max,211.7183,5.6355,0.8278,0.7868
